In [2]:
%load_ext kedro.ipython
%reload_kedro
%load_ext autoreload
%autoreload 2

[12/15/24 20:33:34] INFO     Using                                                                  ]8;id=483411;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\framework\project\__init__.py\__init__.py]8;;\:]8;id=322645;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\framework\project\__init__.py#270\270]8;;\
                             'h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\frame                
                             work\project\rich_logging.yml' as logging configuration.                              

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=452955;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=782457;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py#61\61]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=802370;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=108310;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py#63\63]8;;\

                    INFO     Resolved project path as:                                              ]8;id=19695;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=896297;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py#178\178]8;;\
                             s:\___Studium\Bachelor_Arbeit\ba_env\bundesliga.                                      
                             To set a different path, run '%reload_kedro <project_root>'                           

[12/15/24 20:33:35] INFO     Kedro project bundesliga                                               ]8;id=236294;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=139481;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py#144\144]8;;\

...

[12/15/24 20:33:36] INFO     Registered line magic 'run_viz'                                        ]8;id=427183;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=480818;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py#151\151]8;;\

                    INFO     Resolved project path as:                                              ]8;id=952942;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=386182;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py#178\178]8;;\
                             S:\___Studium\Bachelor_Arbeit\ba_env\bundesliga.                                      
                             To set a different path, run '%reload_kedro <project_root>'                           

                    INFO     Kedro project bundesliga                                               ]8;id=924106;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=943058;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=131103;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=872272;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py#145\145]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=200345;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=638964;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\ipython\__init__.py#151\151]8;;\

In [ ]:
from collections import OrderedDict

import numpy as np
import pandas as pd

GOALS_HOME = "FTHG"
GOALS_AWAY = "FTAG"
df = catalog.load("D1_24-25")


def _get_teams(df: pd.DataFrame) -> np.array:
    team1 = df["HomeTeam"].unique().astype("U")
    team2 = df["AwayTeam"].unique().astype("U")
    teams = np.unique(np.concatenate((team1, team2)))

    assert len(teams) == 18
    # nb_teams = len(teams)

    return teams


def _get_teams_new(df: pd.DataFrame) -> np.array:
    teams, uniques = pd.factorize(
        df[["HomeTeam", "AwayTeam"]].values.flatten(), sort=True
    )

    return teams, uniques


def _build_team_lexicon(teams: np.array) -> pd.DataFrame:
    team_indices = OrderedDict()
    for i, t in enumerate(teams):
        team_indices[t] = i
    lex = pd.DataFrame(list(team_indices.items()), columns=["team", "index"])
    lex.set_index("team", inplace=True)
    return lex


def _get_goal_results(df: pd.DataFrame, team_indices: pd.DataFrame):
    home_goals = list()
    away_goals = list()
    for _, r in df.iterrows():
        home_team = r.HomeTeam

        away_team = r.AwayTeam
        home_goals.append(
            (
                team_indices.loc[home_team, "index"],
                team_indices.loc[away_team, "index"],
                r[GOALS_HOME],
            )
        )
        away_goals.append(
            (
                team_indices.loc[home_team, "index"],
                team_indices.loc[away_team, "index"],
                r[GOALS_AWAY],
            )
        )

    return home_goals, away_goals


def _vectorized_data(home_goals_, away_goals_) -> pd.DataFrame:
    home_id = np.array([hg[0] for hg in home_goals_])
    away_id = np.array([hg[1] for hg in home_goals_])
    home_goals = np.array([hg[2] for hg in home_goals_])
    away_goals = np.array([ag[2] for ag in away_goals_])
    toto = np.where(
        home_goals == away_goals, 0, np.where(home_goals > away_goals, 1, 2)
    )
    vectorized_data = pd.DataFrame(
        {
            "home_id": home_id,
            "away_id": away_id,
            "home_goals": home_goals,
            "away_goals": away_goals,
            "toto": toto,
        }
    )

    return vectorized_data


def preprocess_league_data(df: pd.DataFrame) -> pd.DataFrame:
    teams = _get_teams(df)
    team_indices = _build_team_lexicon(teams=teams)
    home_goals_, away_goals_ = _get_goal_results(df=df, team_indices=team_indices)

    # list of tuples
    # (home_team-index, away_team_index, scored_goals of home team resp. away team)

    return _vectorized_data(home_goals_, away_goals_), team_indices


def create_model_input_data(df: pd.DataFrame) -> pd.DataFrame:
    # merge if we use more then D1 Bundesliga data
    model_input_table = df
    return model_input_table

[12/15/24 13:24:56] INFO     Loading data from D1_24-25 (CSVDataset)...                         ]8;id=655458;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=541301;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\io\data_catalog.py#389\389]8;;\

In [19]:
pre = preprocess_league_data(df)
pre[0].head()



,home_id,away_id,home_goals,away_goals,toto
0,16,1,0,4,2
1,0,11,4,4,0
2,9,7,1,2,2
3,10,13,3,2,1
4,14,2,5,0,1


In [20]:
pre[0][['home_goals','away_goals']]

,home_goals,away_goals
0,0,4
1,4,4
2,1,2
3,3,2
4,5,0
...,...,...
301,2,1
302,4,0
303,2,1
304,4,1


In [76]:
df[["HomeTeam", "AwayTeam", "FTHG", "FTAG"]]["HomeTeam"][10:14]


10       Bochum
11    Darmstadt
12      FC Koln
13     Freiburg
Name: HomeTeam, dtype: object

In [11]:
teams = _get_teams(df)
team_indices = _build_team_lexicon(teams=teams)
home_goals_, away_goals_ = _get_goal_results(df=df, team_indices=team_indices)

In [17]:
np.unique(pre[0][["home_id"]].values)


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17], dtype=int64)

In [77]:
[print(hg) for hg in home_goals_ if hg[0] == 10 and hg[1] == 13]
[print(hg) for hg in away_goals_ if hg[0] == 10 and hg[1] == 13]
[print(hg) for hg in home_goals_ if hg[0] == 13 and hg[1] == 10]
[print(hg) for hg in away_goals_ if hg[0] == 13 and hg[1] == 10]

(10, 13, 3)
(10, 13, 2)
(13, 10, 2)
(13, 10, 3)


[None]

In [3]:
_vectorized_data(home_goals_, away_goals_)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 _vectorized_data(home_goals_, away_goals_)                                                   │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name '_vectorized_data' is not defined

In [ ]:
pre = preprocess_league_data(df)

d = pre[0]
print(d[(d["home_id"] == 10) & (d["away_id"] == 13)])
print(d[(d["home_id"] == 13) & (d["away_id"] == 10)])

   home_id  away_id  home_goals  away_goals  toto
3       10       13           3           2     1
     home_id  away_id  home_goals  away_goals  toto
157       13       10           2           3     2


In [58]:
def _get_teams2(df: pd.DataFrame) -> np.array:
    teams, uniques = pd.factorize(
        df[["HomeTeam", "AwayTeam"]].values.flatten(), sort=True
    )

    return teams, uniques


_get_teams2(df)


(
    array([16,  1,  0, 11,  9,  7, 10, 13, 14,  2, 17,  8,  4,  6, 15, 12,  5,
        3, 13, 14,  2,  4,  3, 15,  6, 17,  7, 16,  8,  9, 11, 10, 12,  5,
        1,  0,  4,  8,  0,  2,  9, 17, 10,  3, 14,  7, 16, 12, 11,  1,  5,
        6, 15, 13,  1, 10,  6,  9,  7,  4, 12, 14, 13,  0, 17, 15,  2,  5,
        8, 16,  3, 11, 14,  3,  0, 12,  1,  2,  4, 17, 11, 13, 15,  9, 16,
        6, 10,  8,  5,  7,  9,  4,  2, 11,  6, 14,  8, 15, 12, 10, 17,  5,
       13,  1,  3, 16,  7,  0, 11, 12,  0,  3,  4, 15, 13,  2, 14, 17, 16,
        9, 10,  6,  1,  7,  5,  8,  4, 16,  3, 13,  7,  2,  9,  5, 15, 14,
       17, 10, 12,  1,  6, 11,  8,  0,  2, 12,  0, 17,  1,  3, 11,  8, 14,
        9, 16, 15, 13,  6,  5,  4, 10,  7,  3,  2,  6,  0,  7, 11,  9, 10,
       12, 13, 15,  5,  4,  1, 17, 16,  8, 14, 11, 17,  0,  9,  1,  8,  3,
       12, 14,  4,  2,  6, 10, 15, 16,  5, 13,  7,  6,  1,  4, 11,  7,  3,
       15,  0, 16, 10, 17, 13,  5, 14,  8,  2,  9, 12,  3,  6,  2, 17, 11,
        9, 13,  8,

In [5]:
# %reload_kedro
from modelbuilder import FootballModel

model = FootballModel()
model_input = catalog.load("model_input_table")
x_data = model_input[["home_id", "away_id"]]
y_data = model_input[["home_goals", "away_goals"]]
y_data_series = y_data.apply(lambda row: (row["home_goals"], row["away_goals"]), axis=1)

model.fit(X=x_data, home_goals=y_data["home_goals"], away_goals=y_data["away_goals"])

[12/14/24 14:45:50] INFO     Loading data from model_input_table (ParquetDataset)...            ]8;id=81715;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=168280;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\io\data_catalog.py#389\389]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 x_data = model_input[["home_id", "away_id"]]                                                │
│    7 y_data = model_input[["home_goals", "away_goals"]]                                          │
│    8                                                                                             │
│ ❱  9 model.fit(X=x_data, home_goals=y_data["home_goals"], away_goals=y_data["away_goals"])       │
│   10                                                                                             │
│                                                                                                  │
│ h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\pymc_experimental\model_builder.py:507 │
│ in fit                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: FootballModel._generate_and_preprocess_model_data() missing 1 required positional argument: 'away_goals'

In [4]:
catalog.load("model_input_table")

[12/15/24 20:33:52] INFO     Loading data from model_input_table (ParquetDataset)...            ]8;id=888410;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=874488;file://h:\Programs\Anaconda\envs\.conda_ba_env\Lib\site-packages\kedro\io\data_catalog.py#389\389]8;;\

,home_id,away_id,home_goals,away_goals,toto
0,16,1,0,4,2
1,0,11,4,4,0
2,9,7,1,2,2
3,10,13,3,2,1
4,14,2,5,0,1
...,...,...,...,...,...
301,10,0,2,1,1
302,14,11,4,0,1
303,15,7,2,1,1
304,16,2,4,1,1


In [11]:
y_data_series = y_data.apply(lambda row: (row["home_goals"], row["away_goals"]), axis=1)

y_data_series


0      (0, 4)
1      (4, 4)
2      (1, 2)
3      (3, 2)
4      (5, 0)
        ...  
301    (2, 1)
302    (4, 0)
303    (2, 1)
304    (4, 1)
305    (1, 3)
Length: 306, dtype: object

In [26]:
import pandas as pd

In [32]:
# Unpack the Series of tuples back into two separate Series
home_goals_series, away_goals_series = zip(*y_data_series)
home_goals_series = pd.Series(home_goals_series, name="home_goals")
away_goals_series = pd.Series(away_goals_series, name="away_goals")

In [35]:
y_data_series.values.flatten()


array([(0, 4), (4, 4), (1, 2), (3, 2), (5, 0), (2, 0), (1, 0), (4, 1),
       (1, 0), (5, 1), (1, 1), (1, 4), (1, 2), (1, 0), (2, 3), (0, 3),
       (1, 1), (3, 1), (2, 2), (2, 2), (3, 1), (5, 1), (5, 0), (4, 0),
       (1, 2), (1, 1), (0, 3), (2, 2), (1, 3), (2, 4), (1, 3), (3, 0),
       (2, 1), (1, 1), (4, 2), (3, 3), (3, 1), (2, 1), (7, 0), (1, 0),
       (0, 1), (0, 2), (2, 1), (4, 1), (0, 0), (1, 3), (1, 3), (0, 2),
       (1, 0), (0, 3), (2, 0), (2, 2), (4, 2), (2, 0), (2, 2), (1, 2),
       (4, 2), (0, 0), (3, 1), (2, 3), (3, 0), (3, 0), (2, 0), (1, 0),
       (1, 3), (2, 1), (1, 3), (0, 3), (1, 2), (1, 3), (3, 1), (2, 5),
       (2, 2), (3, 2), (8, 0), (2, 1), (2, 3), (2, 0), (6, 0), (3, 3),
       (2, 1), (1, 2), (1, 1), (3, 3), (2, 3), (2, 0), (0, 3), (0, 4),
       (2, 2), (2, 0), (4, 0), (1, 1), (4, 2), (0, 0), (2, 1), (1, 1),
       (4, 0), (2, 2), (3, 1), (0, 1), (4, 2), (1, 1), (1, 1), (0, 3),
       (2, 1), (1, 2), (0, 0), (1, 1), (0, 1), (3, 1), (2, 1), (2, 1),
     